# 2017-05-15 : running the main openRetina scripts
---

Progress of my work : After some weeks of reverse-engineering the OpenRetina scripts, I understood them enough to work direclty on them. L. Perrinet and I built some testing scripts that allowed me to try these scripts. The first one, with a fully-noised input, was run successfully. I still have some problems to run the second one, that use a local camera as an input.

- [ ] Run the local testing script
    - [ ] Solve the empty capture problem
    
Since the code() function seems to receive empty inputs, the problem may come from the run() function.  
Trying to find the source of that problem.  

    def run(self):
        (...)
        print('dbg_key1', self.frame)
        data = self.code(self.frame)
        (...)
        
Output : 

    dbg_key1 None
    
Then :

    def grab(self):
        (...)
        print('dbg_key1', frame)
        print('dbg_key2', self.cap.read())
        (...)
        
Output : 
    
    dbg_key1 None
    dbg_key2 (False, None)
    
The problem seem to come from the PhotoReceptor() class, as trying to call it alone don't work : 

    from openRetina import openRetina
    from openRetina import PhotoReceptor
    import cv2

    cam = PhotoReceptor(640,480)
    cam.grab()
    
Output : 

    Unable to capture video
    dbg_key1 (False, None)

I had some internet problems that slowed down my work. My computer get randomly disconnected, forcing me to restart it to make it work again.  
The script problem come from the PhotoReceptor class, as : 

    (...)
    if not self.cap.isOpened(): 
        print('Camera is not opened')
        stop
    print('dbgkey1')
    (...)
    
Output :

    Camera is not opened
    
I'm missing something. While my own *camera.py* and the official *openRetina* PhotoReceptor classes are fairly similar, the openRetina one don't work. 

    Before a downscale of 1 dim1 : 480, dim2 : 640
    Using OpenCV
    After a downscale of 1, dim1 : 0.0, dim2 : 0.0
    
The null dimensions surely come from the closed camera. Why is the camera not opening?  

- [ ] Run the local testing script
    - [ ] Solve the non-opening camera problem

OpenCV documentation suggested to use the following function to force it, but it doesn't change anything (see : http://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_gui/py_video_display/py_video_display.html#capture-video-from-camera) :

    self.cap.open(0)
    
When used through Cheese (capturing program), the camera open and record just fine.  
This command line give informations about the devices connected to the computer's USB bus, including the camera (first one to appear, see : https://help.ubuntu.com/community/Webcam) :

    lsusb -v
    
Even when using a simple script that I already used and that was working, the camera don't launch and no error is given :

    import cv2

    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print('Cam not opnd')
    while(True):
        ret,frame = cap.read()
        cv2.imshow('frame', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

I'm using the last openCV version (3.2.0.7), maybe that's what is causing that bug. But the VideoCapture is still called the same way than the previous versions I used (API of the function for that version : http://docs.opencv.org/3.0-beta/modules/videoio/doc/reading_and_writing_video.html?highlight=videocapture#videocapture-videocapture).  
    The fact that the camera isn't opening on my personnal computer that use the same openCV version reinforce the idea that it is linked to the openCV version used (as suggested here : http://answers.opencv.org/question/88857/videocapture-is-not-working-in-opencv-300-or-the-310-in-windows-7/ and suggested by lot of recent questions on internet about the same issue, eg.: http://answers.opencv.org/question/136678/opencv-error-assertion-failed-sizewidth0-sizeheight0-in-cvimshow-python/, https://github.com/opencv/opencv/issues/8471 and https://github.com/opencv/opencv/issues/8358).  
I need to ask L. Perrinet which version is installed on his computer, since the scripts worked on it.  
All these links suggest that the issue is caused by the version 3.2.x, so I tried to downgrade to 3.1.x, the previous version :

    pip3 uninstall opencv-python
    pip install opencv-python==3.1.0 --user
    
But it didn't solved the issue.  
I'm working on that issue since this morning and I'm still not able to solve it.  I'll talk with L. Perrinet about that.

In the meantime, I tried to run *2017-05-15_using_openretina* and *2017-05-15_using_openretina_server*.

- [X] Run the openRetina script

It works! A window is opening and show several frames captured, with a red/blue color coding on a black background, as programmed by the openRetina script.  
Still, several non-critical errors are showing, that may decrease the script performance.

    *** Pi side ***
    
    mmal: mmal_vc_port_enable: failed to enable port vc.null_sink:in:0(OPQV): ENOSPC
    mmal: mmal_port_enable: failed to enable connected port (vc.null_sink:in:0(OPQV))0x16dcd70 (ENOSPC)
    mmal: mmal_connection_enable: output port couldn't be enabled
    
    💀  Could not load visualisation
    
    Traceback (most recent call last):
      File "2017-05-15_using_openretina.py", line 13, in <module>
        output=['stream'] # output: can be stream, display, ...
      File "/home/pi/openRetina/src/openRetina.py", line 146, in __init__
        self.camera = PhotoReceptor(self.w, self.h)
      File "/home/pi/openRetina/src/openRetina.py", line 46, in __init__
        self.cap = picamera.PiCamera(cam_id)
      File "/usr/lib/python3/dist-packages/picamera/camera.py", line 433, in __init__
        self._init_preview()
      File "/usr/lib/python3/dist-packages/picamera/camera.py", line 513, in _init_preview
        self, self._camera.outputs[self.CAMERA_PREVIEW_PORT])
      File "/usr/lib/python3/dist-packages/picamera/renderers.py", line 558, in __init__
        self.renderer.inputs[0].connect(source).enable()
      File "/usr/lib/python3/dist-packages/picamera/mmalobj.py", line 2212, in enable
        prefix="Failed to enable connection")
      File "/usr/lib/python3/dist-packages/picamera/exc.py", line 184, in mmal_check
        raise PiCameraMMALError(status, prefix)
    picamera.exc.PiCameraMMALError: Failed to enable connection: Out of resources


# 2017-05-16 : capturing a video file with the Pi and sending it to a server
---

These errors seem to be linked to picamera's mmalobj module (complete API : http://picamera.readthedocs.io/en/latest/api_mmalobj.html). They don't show at every run.  

Now that the openRetina script is correctly running, I can begin to apply modifications to it. In accordance with L. Perrinet, I'll insert a simple script to capture and save a video file and then modify the script to allow it to work with that file, instead of a stream. That'll allow to bypass the networking limitations, like bandwidth consumption.  

I'll then capture a video with a stable background and a moving object (eg. a hand), so some of the pixels will change value while other will keep the same over time. The objective is to modify the program to only code (send to server) information about the changing pixels instead of coding the luminance contrast as it does now.

About the poster that'll act as my final presentation, L. Perrinet thinks that it may be better to show what I've learned (python classes, git and ssh use, raspberry manipulation, biomimetism, ...) instead of my results.

- [X] Update openRetina script to get more informative prints
- [X] Write a simple script to capture a video to a file

More informations about the h264 video files format : https://en.wikipedia.org/wiki/H.264/MPEG-4_AVC  
At first, making a local (on the Pi) video capture script without any networking to make trials (script inspired by http://picamera.readthedocs.io/en/latest/recipes1.html#recording-video-to-a-file) :

    import picamera

    picam = picamera.PiCamera
    picam.resolution = (640,480)              #Defining the camera resolution
    picam.start_recording('video_picam.h264') #Start recording in 'filename.filetype'
    picam.wait_recording(10)                  #Better integrated time.wait() that allow to continually check for errors
    picam.stop_recording()                    #Stop recording

But that script output an error (not present on the internet example, my be induced by the that its the Pi that run the script) : 

    picam.start_recording('video_picam.h264') #Start recording in 'filename.filetype'
    TypeError: start_recording() missing 1 required positional argument: 'output'
    
After modifying the concerned line a bit, and even if the output argument is the only non-optionnal (see API here : http://picamera.readthedocs.io/en/latest/api_camera.html#picamera.PiCamera.start_recording), the script still asks for more 'missing' arguments : 

    (...)
    picam.start_recording(output='video_picam', format='h264') #Start recording in 'filename.filetype'
    (...)

Output : 

    picam.start_recording(output='video_picam', format='h264') #Start recording in 'filename.filetype'
    TypeError: start_recording() missing 1 required positional argument: 'self'
    
That was an human error, and I simply solved the problem by going from :

    (...)
    picam = picamera.PiCamera
    (...)

To : 

    (...)
    picam = picamera.PiCamera()
    (...)
    
- [ ] Send the video file to a server

Trying to write duo of client/server scripts to send/rcv the video file that was recorded. I do this to try my parameters before blend it in the main openRetina script. By I'm facing a newtorking problem, as the Pi-sided script output :

    (...)
    socket.bind("tcp://*:5555")
    File "socket.pyx", line 444, in zmq.backend.cython.socket.Socket.bind (zmq/backend/cython/socket.c:4124)
    File "checkrc.pxd", line 21, in zmq.backend.cython.checkrc._check_rc (zmq/backend/cython/socket.c:6103)
    zmq.error.ZMQError: Address already in use
    
And the command (more informations here : http://stackoverflow.com/questions/19159771/recovering-from-zmq-error-zmqerror-address-already-in-use) : 

    netstat -ltnp tcp

Output : 

    (...)
    Active Internet connections (only servers)
    Proto Recv-Q Send-Q Local Address           Foreign Address         State       PID/Program name
    tcp        0      0 0.0.0.0:5555            0.0.0.0:*               LISTEN      2006/python3    
    tcp        0      0 0.0.0.0:5556            0.0.0.0:*               LISTEN      1849/python3    
    (...)

A connection can be manually closed with the command : 

    kill -9 <pid>

That issue can be resolved by adding encapsulation (try/except/finally) loops in the script.

Since I already did it, I first tried to build a network using the REQ/REP zmq's parameters, but other options can be tried : http://zguide.zeromq.org/py:fileio1, http://zguide.zeromq.org/py:fileio3, http://zguide.zeromq.org/py:fileio3  

# 2017-05-17 : correctly encoding videos to send them to the server
---
I still have problems with my internet connection around 9, that makes me restart the Mac computer.  

The Pi/computer connection is correctly initiated, but the file is not send, as that error message is outputed :

    File "2017-05-16_capture_videos_with_openretina_client.py", line 17, in <module>
        socket.send(Vid)
      File "socket.pyx", line 574, in zmq.backend.cython.socket.Socket.send (zmq/backend/cython/socket.c:5390)
      File "socket.pyx", line 621, in zmq.backend.cython.socket.Socket.send (zmq/backend/cython/socket.c:5157)
      File "socket.pyx", line 168, in zmq.backend.cython.socket._send_copy (zmq/backend/cython/socket.c:2021)
      File "buffers.pxd", line 200, in buffers.asbuffer_r (zmq/backend/cython/socket.c:7091)
      File "buffers.pxd", line 151, in buffers.asbuffer (zmq/backend/cython/socket.c:6523)
    TypeError: <_io.TextIOWrapper name='video_picam' mode='r' encoding='UTF-8'> does not provide a buffer interface.

More informations about transferring files with zmq : http://zguide.zeromq.org/php:chapter7#Transferring-Files  
(More informations about the raise statement : https://docs.python.org/3/tutorial/errors.html#raising-exceptions)  
(Some informations about the assert statement, equivalent of a "if not" line : http://stackoverflow.com/questions/5142418/what-is-the-use-of-assert-in-python)  

Trying to insert the zmq simplest method to send files (see : http://zguide.zeromq.org/py:fileio1) into the *_capture_videos_with_openretina* scripts.  

After insetring it and correcting some human-made breaking errors, I face that error :

    data = file.read(CHUNK_SIZE)
    File "/usr/lib/python3.4/codecs.py", line 313, in decode
    (result, consumed) = self._buffer_decode(data, self.errors, final)
    UnicodeDecodeError: 'utf-8' codec can't decode byte 0xac in position 8: invalid start byte
    
- [ ] Send the video file to a server
    - [X] Solve the encoding format problem

So there seems it is a data encoding problem, but it looks weird that the error is only triggered at the 8th position.  
I'll try to change the recording format, decided at :

    (...)
    cam.start_recording(output='video_picam2', format='h264')
    (...)
    
When switching format to rgb, and even without changing the recording time, the camera stayed opened for a longer time than usual. Maybe the rgb format -a raw 24-bit format- is heavier so the program is severly slowed down? Anyway the error stayed.  

I tried to explicilty tell the encoding format when opening the video file (as suggested here : http://stackoverflow.com/questions/19699367/unicodedecodeerror-utf-8-codec-cant-decode-byte) in that line :

    (...)
    file = open("video_picam","r", encoding="rgb")
    (...)
        
The script now output that error : 

    (...)
    file = open("video_picam","r", encoding="rgb")
    LookupError: unknown encoding: rgb
    
I modified again that line to make the script open the file as bytes (more informations about open() : https://docs.python.org/3/tutorial/inputoutput.html#reading-and-writing-files) :

    (...)
    file = open("video_picam","rb", encoding="rgb")
    (...)
    
The error changed again, asking me to reverse one of my previous actions : 

    (...)
    file = open("video_picam","rb", encoding="rgb")
    ValueError: binary mode doesn't take an encoding argument

I've almost reached my goal. The computer-sided script output : 

    Connected
    Sending request
    3 chuncks received, 418001 bytes
    Connection closed
 
But also an error, easily corrected : 

    (...)
    NameError: name 'pipe' is not defined
    
The chunks seems correctly transferred, but they're not built into a file so I can't work on them later. I may need to concatenate them into one.  
Problem : The Pi-sided script isn't closing after the computer-sided one did.  

- [ ] Send the video file to a server
    - [X] Solve the non-closing script problem

The script was trapped in a infinite *while* loop. Solved it by adding a *break* line at the end of it :

    (...)
    if not data: break

- [ ] Send the video file to a server
    - [ ] Rebuild the received file chunks into a file 
    
Note that the script is simple and don't control at all the information flow, so it face an overflow risk for bigger files. Its possible to later modify that script by following the zmq documentation (here : http://zguide.zeromq.org/php:chapter7#Transferring-Files).

# 2017-05-18
---
I talked with V. Boutin about my work and he asked me why did I built a script to record a video in a file then send it to the server instead of working with a stream like before to make the server build the video file. He thinks the method I used is much slower and more energy consuming.  
I don't agree with that point. Of course my method is slower than using a direct stream, but it allow to have a copy of the video file on both the Pi and the computer so later I may implement a way to compare the pre- and post-processing files, plus it seems easier to do some pre-processing on the video files before sending it to the computer if we have a copy of it in the Pi.  
Regarding the energy consumption, I think it is smaller with my method because the video recording and sending are separated process, allowing to record the video only one time and to do some pre-processing/sending the video whenever we want or have free resources on the Pi.

Idea for later : it may be interesting to add a way to delete the video file in the Pi, to not overfill its memory.

# Outline
---

- [ ] Run the local testing script
    - [ ] Solve the empty capture problem
    - [ ] Solve the non-opening camera problem
- [X] Run the openRetina script
- [X] Update openRetina script to get more informative prints
- [X] Write a simple script to capture a video to a file
- [ ] Send the video file to a server
    - [X] Solve the encoding format problem
    - [X] Solve the non-closing script problem
    - [ ] Rebuild the received file chunks into a file
- [ ] Capture a video with a stable background and moving object
- [ ] [OPT]  Add a way to delete the video file on the Pi after sending and processing are done.